In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
# import argparse
# from collections import namedtuple
import glob


In [ ]:
nc_files = glob.glob('trih*.nc')
obs_files = glob.glob('*.obs')
if len(obs_files) == 1:
    # nc_file = nc_files[0]
    obs_file = obs_files[0]

In [ ]:
def extract_station_name(file_path):
    obs_station_name = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            obs = line.split()[0]
            obs_station_name.append(obs.strip())
    return obs_station_name

obs_station_name = extract_station_name(obs_file)
print(obs_station_name)

In [ ]:
model_datasets = {}
for nc_file in nc_files:
    model_dataset = xr.open_dataset(nc_file)
    model_datasets[nc_file.split('-')[-1].split('.')[0]] = model_dataset
model_datasets.keys()

In [ ]:
def Read_tek_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    content_without_header = '\n'.join(content.split('\n')[5:])

    df = pd.read_csv(io.StringIO(content_without_header), 
                    delim_whitespace=True, 
                    names=['Date', 'Time', 'WL'])

    def pad_time(time_str):
        return time_str.zfill(6)

    df['Time'] = df['Time'].astype(str).apply(pad_time)
    df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str), format='%Y%m%d %H%M%S')
    df = df.drop(columns=['Date', 'Time'])
    df = df[['DateTime', 'WL']]

    return df

def make_obs_df(obs_station_name):
    obs_df = pd.DataFrame()
    obs_df['Time'] = Read_tek_file(f"{obs_station_name[0]}.tek")['DateTime']
    for name in obs_station_name:
        df = Read_tek_file(f"{name}.tek")['WL']
        obs_df[name] = df

    return obs_df

obs_df = make_obs_df(obs_station_name)
# obs_df["Time"].min()

In [ ]:
model_df_s = {}
for nc_name, model_dataset in model_datasets.items():
    model_df = pd.DataFrame()
    model_df['Time'] = model_dataset.time
    for i, name in enumerate(obs_station_name):
        model_df[name] = model_dataset['ZWL'].to_numpy()[:, i]
    model_df_s[nc_name] = model_df

# model_df_s.keys()
# model_df_s["coarser20180701"]

In [ ]:
def clipped_dataframe(model_df, obs_df):
    start_time = max(model_df['Time'].min(), obs_df['Time'].min())
    end_date = min(model_df['Time'].max(), obs_df['Time'].max())
    print(start_time, end_date)

    model_df = model_df[(model_df['Time'] >= start_time) & (model_df['Time'] <= end_date)]
    obs_df = obs_df[(obs_df['Time'] >= start_time) & (obs_df['Time'] <= end_date)]
    # print(model_df.shape, obs_df.shape)
    return model_df, obs_df

obs_df_s = {}
for nc_name, model_df in model_df_s.items():
    # print(nc_name)
    model_df, obs_df_temp = clipped_dataframe(model_df, obs_df)
    model_df_s[nc_name] = model_df
    obs_df_s[nc_name] = obs_df_temp

# obs_df_s
# model_df_s

In [ ]:
def interpolate(obs_df, model_df):
    min_interval = min(obs_df['Time'].diff().min(), model_df['Time'].diff().min())

    new_time_index = pd.date_range(start=obs_df['Time'].iloc[0],
                                end=obs_df['Time'].iloc[-1],
                                freq=min_interval)

    model_interpolated = model_df.set_index('Time').reindex(new_time_index).interpolate()
    obs_interpolated = obs_df.set_index('Time').reindex(new_time_index).interpolate()
    # print(model_interpolated.shape, obs_interpolated.shape)
    # Reset the index and set the name as Time
    model_interpolated = model_interpolated.reset_index().rename(columns={'index': 'Time'})
    obs_interpolated = obs_interpolated.reset_index().rename(columns={'index': 'Time'})

    return model_interpolated, obs_interpolated


model_interpolated_s = {}
obs_interpolated_s = {}

for nc_name in model_df_s.keys():
    print(nc_name)
    model_interpolated, obs_interpolated = interpolate(obs_df_s[nc_name], model_df_s[nc_name])
    # print(obs_interpolated)
    # print(model_interpolated)
    model_interpolated_s[nc_name] = model_interpolated
    obs_interpolated_s[nc_name] = obs_interpolated
    # break
# model_df_s.keys()

In [ ]:
# def plot(obs_station_name, model_interpolated, obs_interpolated, ):
#     fig, axs = plt.subplots(2, 2, figsize=(12, 10))
#     axs = axs.ravel()  # Flatten the 2x2 array to make indexing easier

#     for ax in axs:
#         ax.clear()  # Clear the axes to prevent multiple legends
        
#     for i, name in enumerate(obs_station_name):
#         model = model_interpolated[name]
#         obs = obs_interpolated[name]

#         axs[i].plot(model, label=f"model", color='blue', linewidth=1)
#         axs[i].plot(obs, label=f"observed", linestyle='none', marker='o', markersize=2, color="red")
#         # axs[i].plot(obs_interpolated[name], label=f"observed - {name}", linestyle='none', marker='o', markersize=2)

#         axs[i].set_title(name)
#         # axs[i].set_xlabel('X-axis')
#         axs[i].set_ylabel('WL')
#         axs[i].tick_params(axis='x', rotation=40)
#         axs[i].legend()

#     plt.tight_layout()
#     plt.show()


# plot(obs_station_name, model_interpolated, obs_interpolated)

In [49]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import r2_score
quality_indices = {}

def plot(obs_station_name, model_interpolated, obs_interpolated, nc_name):
    fig = make_subplots(rows=2, cols=2, subplot_titles=obs_station_name, 
                        vertical_spacing=0.1, horizontal_spacing=0.09,
                        # column_widths=[0.9, 0.9]
                        )
    # model_interpolated = model_interpolated["Time"]
    # Clip out the first day
    # first_day = model_interpolated.index[0].date()
    # model_interpolated = model_interpolated[model_interpolated.index.date > first_day]

    for i, name in enumerate(obs_station_name):
        row = i // 2 + 1
        col = i % 2 + 1

        model = model_interpolated[name][40:]
        obs = obs_interpolated[name][40:]

        model_tidal_range = model.max() - model.min()
        obs_tidal_range = obs.max() - obs.min()

        model_max = model.max()
        model_min = model.min()
        # obs_max = obs.max()

        # Calculate R2, rmse, bias
        r2 = r2_score(obs.dropna(), model.dropna())
        rmse = np.sqrt(np.mean((model.dropna() - obs.dropna())**2))
        bias = np.mean(model.dropna() - obs.dropna())
        
        if nc_name not in quality_indices:
            quality_indices[nc_name] = {}
        if name not in quality_indices[nc_name]:
            quality_indices[nc_name][name] = {}
        quality_indices[nc_name][name]["R2"] = round(r2, 3)
        quality_indices[nc_name][name]["RMSE"] = round(rmse, 3)
        quality_indices[nc_name][name]["BIAS"] = round(bias, 3)

        fig.add_trace(
            go.Scatter(x=model_interpolated["Time"], y=model, name="model", line=dict(color='blue', width=1), showlegend=(i==0)),
            row=row, col=col
        )
        fig.add_trace(
            go.Scatter(x=obs_interpolated["Time"], y=obs, name="observed", mode='markers', marker=dict(color='red', size=2), showlegend=(i==0)),
            row=row, col=col
        )

        # Add R2, RMS, Bias
        fig.add_annotation(
            xref="x domain", yref="y domain",
            x=1, y=1,
            text=f"R² = {r2:.3f}    RMS = {rmse:.3f}    Bias = {bias:.3f}",
            showarrow=False,
            font=dict(size=15),
            align="right",
            row=row, col=col
        )

        # Add tidal range
        fig.add_annotation(
            xref="x domain", yref="y domain",
            x=1, y=0.92,
            text=f"Model TR: {model_tidal_range:.2f}    Obs TR: {obs_tidal_range:.2f}",
            showarrow=False,
            font=dict(size=15),
            align="right",
            row=row, col=col
        )

        # Add max values
        fig.add_annotation(
            xref="x domain", yref="y domain",
            x=1, y=0.01,
            text=f"Model Max WL: {model_max:.2f}   Model Min WL: {model_min:.2f}",
            showarrow=False,
            font=dict(size=15),
            align="right",
            row=row, col=col
        )
            
        fig.update_yaxes(title_text="WL", row=row, col=col)


    fig.update_layout(height=800, width=1600,
                      plot_bgcolor='lightblue',
                      paper_bgcolor='lightyellow',
                      xaxis=dict(gridcolor='white'),
                      yaxis=dict(gridcolor='white'))
    fig.update_xaxes(gridcolor='white')
    fig.update_yaxes(gridcolor='white')
    # fig.show()

    fig.write_image(f"{nc_name}.png", scale=2)  # scale=2 for higher resolution

for nc_name in model_interpolated_s.keys():
    print(nc_name)
    plot(obs_station_name, model_interpolated_s[nc_name], obs_interpolated_s[nc_name], nc_name)


coarser20180101
coarser20180701
coarser20190101
coarser20190701


In [48]:
data = []
for model, stations in quality_indices.items():
    for station, metrics in stations.items():
        if model[-3] == "1":
            row = {'Season': "Dry Season", 'Station': station}
            row.update(metrics)
            # print(row)
            data.append(row)
        elif model[-3] == "7":
            row = {'Season': "Wet Season", 'Station': station}
            row.update(metrics)
            data.append(row)
        else:
            raise ValueError("Invalid model name")

df = pd.DataFrame(data)

# df = df[['Model', 'Station', 'R2', 'RMSE', 'BIAS']]

df.to_csv('quality_indices.csv', index=False)
